### Load and analyze the dataset

In [1]:
from datasets import load_dataset

dataset_de = load_dataset("nielsr/XFUN", "xfun.de")

c:\Users\Habram\Anaconda3\envs\LayoutXLM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset xfun (C:/Users/Habram/.cache/huggingface/datasets/nielsr___xfun/xfun.de/0.0.0/ace0bc21a86b3d618485f1a8a03978ee46a0a05a10a69465e07593c7645d0804)
100%|██████████| 2/2 [00:00<00:00, 15.50it/s]


In [11]:
dataset_de['train'].features

{'id': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'bbox': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(names=['O', 'B-QUESTION', 'B-ANSWER', 'B-HEADER', 'I-ANSWER', 'I-QUESTION', 'I-HEADER'], id=None), length=-1, id=None),
 'image': Array3D(shape=(3, 224, 224), dtype='uint8', id=None),
 'original_image': Image(decode=True, id=None),
 'entities': Sequence(feature={'start': Value(dtype='int64', id=None), 'end': Value(dtype='int64', id=None), 'label': ClassLabel(names=['HEADER', 'QUESTION', 'ANSWER'], id=None)}, length=-1, id=None),
 'relations': Sequence(feature={'head': Value(dtype='int64', id=None), 'tail': Value(dtype='int64', id=None), 'start_index': Value(dtype='int64', id=None), 'end_index': Value(dtype='int64', id=None)}, length=-1, id=None)}

In [9]:
features = dataset_de["train"].features

for feature in features:
    if feature != 'original_image':
        print('Length of', feature, ':', len(dataset_de['train'][0][feature]))
    print(dataset_de['train'][0][feature])

Length of id : 12
de_train_0_0
Length of input_ids : 512
[873, 106566, 4, 12426, 165, 47036, 22389, 224, 83604, 7, 12, 161240, 1659, 68, 96596, 165, 6, 201580, 6, 138877, 184, 1079, 78767, 9, 1367, 104937, 7, 33357, 68575, 20, 873, 32784, 1276, 23, 69260, 13, 3471, 22389, 6280, 1079, 1667, 177038, 20, 62, 94841, 404, 24993, 15270, 81421, 27420, 5761, 119311, 71317, 17331, 127050, 87021, 15757, 4, 4855, 11627, 4, 222589, 5, 138621, 7, 11627, 93978, 4, 223193, 7, 226845, 138621, 7, 90722, 40902, 5165, 4859, 98989, 884, 731, 53, 4, 182639, 71887, 953, 611, 12825, 158287, 963, 13023, 893, 37738, 15, 234070, 4, 16384, 22389, 4, 2795, 115688, 57292, 4, 48991, 19135, 16, 12298, 36883, 8131, 5, 203068, 3993, 106850, 60592, 27420, 5761, 4, 60592, 4, 4859, 98989, 884, 731, 53, 4, 182639, 71887, 496, 4, 168707, 123048, 5757, 2681, 81929, 9022, 77383, 12, 64086, 59134, 176582, 15, 163612, 4, 4855, 11627, 4, 893, 37738, 4, 13840, 16, 335, 40614, 34083, 104598, 64, 241152, 18917, 615, 70324, 5386, 9

In [10]:
labels = dataset_de['train'].features['labels'].feature.names
id2label = {k:v for k,v in enumerate(labels)}
label2id = {v:k for k,v in enumerate(labels)}
print(id2label)

{0: 'O', 1: 'B-QUESTION', 2: 'B-ANSWER', 3: 'B-HEADER', 4: 'I-ANSWER', 5: 'I-QUESTION', 6: 'I-HEADER'}


### Prepare the dataset for the model

In [12]:
from transformers import AutoProcessor
from transformers import LayoutXLMTokenizer

processor = AutoProcessor.from_pretrained("microsoft/layoutxlm-base", apply_ocr=False)
tokenizer = LayoutXLMTokenizer.from_pretrained("microsoft/layoutxlm-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LayoutXLMTokenizer'. 
The class this function is called from is 'LayoutLMv2TokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [31]:
def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]

def prepare_examples(examples):
  print('itt vagyok')
  images = examples["original_image"]
  print('itt image')
  words = []
  for id in examples['input_ids'][0]:
    words.append(tokenizer.decode(id))
  print('itt words')
  boxes = []
  for box in examples['bbox'][0]:
    boxes.append(unnormalize_box(box, 224, 224))
  print('itt box')
  word_labels = examples['labels']
  print('itt label')

  encoding = processor(images, words, boxes=boxes, word_labels=word_labels, 
                       truncation=True, padding="max_length", max_length=512)

  return encoding

In [32]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

# we need to define custom features for `set_format` (used later on) to work properly
features = Features({
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'image': Array3D(dtype="float32", shape=(3, 224, 224)),
})

train_dataset = dataset_de["train"].map(
    prepare_examples,
    batched=True,
    remove_columns=dataset_de["train"].column_names,
    features=features,
)

Map:   0%|          | 0/189 [00:00<?, ? examples/s]

itt vagyok
itt image
512
['Be', 'zeichnung', ',', 'Ort', 'und', 'Geschäfts', 'nummer', 'des', 'Gericht', 's', ':', 'Erklärung', 'über', 'die', 'persönlichen', 'und', '', 'wirtschaftlichen', '', 'Verhältnis', 'se', 'bei', 'Prozess', '-', 'oder', 'Verfahren', 's', 'kosten', 'hilfe', '-', 'Be', 'lege', 'sind', 'in', 'Kopi', 'e', 'durch', 'nummer', 'iert', 'bei', 'zu', 'fügen', '-', 'A', 'Angaben', 'zu', 'Ihrer', 'Person', 'Julian', 'Herr', 'mann', 'Lehrer', '09.', '08.', '1987', 'ledig', 'Name', ',', 'Vor', 'name', ',', 'ggf', '.', 'Geburt', 's', 'name', 'Beruf', ',', 'Erwerb', 's', 'tätigkeit', 'Geburt', 's', 'datum', 'Familien', 'stand', '55', '232', 'Al', 'ze', 'y', ',', 'Bahnhof', 'straße', '19', '16', '72', '489', '10', '42', 'An', 'schrift', '(', 'Straße', ',', 'Haus', 'nummer', ',', 'Post', 'leit', 'zahl', ',', 'Wohn', 'ort', ')', 'Tags', 'über', 'tel', '.', 'erreichbar', 'unter', 'Nummer', 'Julia', 'Herr', 'mann', ',', 'Julia', ',', '55', '232', 'Al', 'ze', 'y', ',', 'Bahnhof', 's

ValueError: Id not recognized

In [36]:
images = dataset_de["train"]["original_image"][0]
words = []
for id in dataset_de["train"]['input_ids'][0]:
  words.append(tokenizer.decode(id))
boxes = dataset_de["train"]['bbox'][0]
word_labels = dataset_de["train"]['labels']


encoding = processor(images, words, boxes=boxes, word_labels=word_labels, truncation=True, padding="max_length", max_length=512)

ValueError: Id not recognized